# Training -> Prediction

### Updated 2018-09-05
### test_date == prediction date: predict 6-1, then test_date = 6-1 as input
- change: variable reset: test_date 改动 -1天

In [1]:
# coding: utf-8

# # Train to prediction module
# - Optimized for speed
# - Following Feature Preprocessing Module
# 
# ## updated 2018-08-30: 
# - fix bug
# - output training pred_train, true_train
# - output in DataFrame format for faster evaluation speed

# In[21]:


#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 31 13:46:58 2018

@author: yuze
@updated: Ester & Rui
"""
import os
os.chdir('/Users/esterwang/Desktop/Project/Demand Forecasting/demand_forecasting/')

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#%matplotlib inline

from datetime import date, timedelta, datetime, time
from dateutil.relativedelta import relativedelta

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

#from sklearn.grid_search import GridSearchCV
from sklearn.metrics.scorer import make_scorer

from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import lightgbm as lgb
import copy
import pickle

import warnings
warnings.filterwarnings("ignore")

/anaconda3/envs/forecast/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# ## User_defined functions

# In[22]:


def get_formated_index(quantile_list):
    return pd.Float64Index(data=sorted([x/100 for x in quantile_list]), name='Quantile')
    
def reformat(x, quantile_list, pred_len_list):
    temp = pd.DataFrame(np.reshape(x.values, [len(quantile_list), len(pred_len_list)]),
             index=get_formated_index(quantile_list), columns=sorted(pred_len_list))
    return pd.DataFrame(np.sort(temp.values, axis=0), index=temp.index, columns=temp.columns)

def get_rolling_quantile(df, quantile_list, window_list, path="feature_data"):
    """
    Get rolling quantile values
    :param df: a dataframe of input sales data
    :param quantile_list: a list of quantile values
    :param window_list: a list of time windows
    :param path: directory to store feature data files
    :return: save dataframe as csv file to local disk
    """
    if not os.path.isdir(os.path.join(os.getcwd(), path)):
        os.makedirs(path)
    for quantile in quantile_list:
        for window in window_list:
            quantile_key = 'window_{window}_quantile_{quantile}'.format(window=str(window), quantile=str(quantile))
            print quantile_key
            temp = df.T.rolling(window).quantile(float(quantile)/100).T
            temp.to_csv(path+'/'+quantile_key)
    return


def get_rolling_mean(df, window_list, path="feature_data"):
    """
    Get rolling mean values
    :param df: a dataframe of input sales data
    :param window_list: a list of time windows
    :param feature_data: directory to store feature data files
    :return: save dataframe as csv file to local disk
    """
    if not os.path.isdir(os.path.join(os.getcwd(), path)):
        os.makedirs(path)
    for window in window_list:
        mean_key = 'window_{window}_mean'.format(window=str(window))
        print mean_key
        temp = df.T.rolling(window).mean().T
        temp.to_csv(path+'/'+mean_key)
    return


def read_feature_data(quantile_list, quantile_window_list, mean_window_list, path="feature_data"):
    """
    read feature_data from path
    :param quantile_list: a list of quantile values
    :param window_list: a list of window values
    :param path: directory of the feature data
    :return: two dictionaries of dataframes
    """
    quantile_feature = dict()
    mean_feature = dict()
    for window in quantile_window_list:
        for quantile in quantile_list:
            quantile_key = 'window_{window}_quantile_{quantile}'.format(window=str(window), quantile=str(quantile))
            print 'Reading ', quantile_key 
            df = pd.read_csv(path+'/'+quantile_key, index_col=0)
            df.columns = pd.to_datetime(df.columns)
            quantile_feature[quantile_key] = df
            
        
    for window in mean_window_list:
        mean_key = 'window_{window}_mean'.format(window=str(window))
        print 'Reading ', mean_key
        df = pd.read_csv(path+'/'+mean_key, index_col=0)
        df.columns = pd.to_datetime(df.columns)
        mean_feature[mean_key] = df
           
    return quantile_feature, mean_feature


def get_quantile_value(quantile, window, quantile_feature, train_date):
    """

    :param quantile:
    :param window:
    :param quantile_feature:
    :param train_date: datetime object
    :return:
    """
    train_date = (train_date-timedelta(1)).strftime("%Y-%m-%d 00:00:00")
    quantile_key = 'window_{window}_quantile_{quantile}'.format(window=str(window), quantile=str(quantile))
    data = quantile_feature.get(quantile_key, None)
    if data is None:
        return
    return data.loc[:, train_date]


def get_mean_value(window, mean_feature, train_date):
    """

    :param window:
    :param mean_feature:
    :param train_date:
    :return:
    """
    train_date = (train_date-timedelta(1)).strftime("%Y-%m-%d 00:00:00")
    mean_key = 'window_{window}_mean'.format(window=window)
    data = mean_feature.get(mean_key, None)
    if data is None:
        return
    return data.loc[:, train_date]


def get_back_timespan(df, dt, minus, period):
    date_common = pd.date_range(start=dt-timedelta(days=minus), periods=period) & df.columns
    len_common = len(date_common)
    date_diff = pd.date_range(start=dt-timedelta(days=(minus+30)), periods=(period-len_common))
#     return df[pd.date_range(start=dt-timedelta(days=minus+period-len_common), periods=period)]
    return df[date_diff|date_common]


def get_forward_timespan(df, dt, period):
    date_common = pd.date_range(start=dt, periods=period) & df.columns
    len_common = len(date_common)
    if period > len_common:
        date_diff = pd.date_range(start=dt+timedelta(30), periods=period)[-(period-len_common):]
    else: date_diff = pd.to_datetime([])
    return df[date_common|date_diff]


def get_summary_stats(df, train_date, model):
    """

    :param df:
    :param train_date:
    :param model:
    :return:
    """
    X = pd.DataFrame()
    window_list = [[3, 7, 14, 30, 60, 140], [7, 14, 30, 60, 140]] 
    for window in window_list[0]:
        tmp = get_back_timespan(df, train_date, window, window)
        X['diff_%s_mean_2' % window] = tmp.diff(axis=1).mean(axis=1)
        X['mean_%s_decay_2' % window] = (tmp * np.power(0.9, np.arange(window)[::-1])).sum(axis=1)
        X['median_%s_2' % window] = tmp.median(axis=1).values # median is exactly 50th quantile
        X['min_%s_2' % window] = tmp.min(axis=1)
        X['max_%s_2' % window] = tmp.max(axis=1)
        X['std_%s_2' % window] = tmp.std(axis=1)
        
    for window in window_list[1]:
        tmp = get_back_timespan(df, train_date, window, window)
        X['has_sales_days_in_last_%s' % window] = (tmp > 0).sum(axis=1)
        X['last_has_sales_day_in_last_%s' % window] = window - ((tmp > 0) * np.arange(window)).max(axis=1)
        X['first_has_sales_day_in_last_%s' % window] = ((tmp > 0) * np.arange(window, 0, -1)).max(axis=1)

    return X


def get_label(df, train_date, quantile, pred_len, model, is_train):
    if not is_train: return
    tmp = get_forward_timespan(df, train_date, pred_len)
    if model =='M2Q':
        y = np.sqrt(np.sum(tmp, axis=1))  # Yuze Quantile regression #^(1/2) smooth
    else:
        y_sum = []
        pred_len_tmp = int(np.max([pred_len, 31]))
        for i in range(pred_len_tmp):
            y_sum.append(np.sum(get_back_timespan(df, train_date, pred_len_tmp // 2 - i, pred_len), axis=1))
        y = np.sqrt(np.percentile(y_sum, quantile, axis=0)).transpose()
    return y


def get_features(quantile, quantile_window_list, mean_window_list, train_date, quantile_feature, mean_feature): # done
    """

    :param quantile_list:
    :param window_list:
    :param train_date:
    :param quantile_feature:
    :param mean_feature:
    :return:
    """
    X = pd.DataFrame()
    for window in quantile_window_list:
        quantile_column = 'window_{window}_quantile_{quantile}'.format(window=window, quantile=quantile) #TODO: change this hard coded thing
        quantile_column_name = 'q_{window}_2017'.format(window=window)
        X[quantile_column_name] = get_quantile_value(quantile, window, quantile_feature, train_date)
        
    for window in mean_window_list:
        mean_column = 'window_{window}_mean'.format(window=window)
        mean_column_name = 'mean_{window}_2017'.format(window=window)
        X[mean_column_name] = get_mean_value(window, mean_feature, train_date)
    return X


def prepare_dataset(df, train_date, quantile, quantile_window_list, mean_window_list, pred_len, model, is_train):
    quantile_mean_data = get_features(quantile, quantile_window_list, mean_window_list, train_date, quantile_feature, mean_feature)
    summary_stats_data = get_summary_stats(df, train_date, model)
    y = get_label(df, train_date, quantile, pred_len, model,is_train)
    all_data = [quantile_mean_data, summary_stats_data]
    data = pd.concat(all_data, axis=1)
    return data, y


def get_train_date(test_date, pred_len, is_far, rolling_num, rolling_span, time_back_num):
    if is_far: #Yuze hardcode jump parameters 3
        train_date = test_date - relativedelta(months=12) - relativedelta(days = rolling_span * time_back_num) # set earliest jump start 
    else: # ensure NOT TOO NEAR
        pred_len_tmp = int(np.max([pred_len,31]))
        train_date = test_date - relativedelta(days= 2 * pred_len_tmp + (rolling_num-1) * rolling_span)  ##########
    return train_date


def prepare_training_data(df, quantile, is_far, pred_len, model, test_date, mean_window_list, quantile_window_list, is_train, time_back_num = 2, rolling_num=5, rolling_span=7):
    if is_train:
        X = [None]*rolling_num
        y = [None]*rolling_num
    
        train_date = get_train_date(test_date, pred_len, is_far, rolling_num, rolling_span, time_back_num)
        for i in range(rolling_num):
            delta = timedelta(days=rolling_span * i)  # Yuze jumps around, from earliest to latest
            X_tmp, y_tmp = prepare_dataset(df, train_date + delta, quantile, quantile_window_list, mean_window_list, pred_len, model, is_train)
            X[i] = X_tmp
            y[i] = y_tmp
        X = pd.concat(X, axis=0)
        y = [i for yy in y for i in yy]
        return X, y
    
    else: # 2018-08-30 bug fixed
        X, _ = prepare_dataset(df, test_date , quantile, quantile_window_list, mean_window_list, pred_len, model, is_train)
        return X

    


# In[ ]:


def sort_quantile(df):
    # Make sure higher prediction at higher quantiles - re-arrange
    temp = df.reindex_axis(sorted(df.columns, key=lambda x: (x[0],x[1])), axis=1)
    temp_sort = list(map(lambda x: sorted(x),  temp.values))
    temp_df = pd.DataFrame(temp_sort, index = df.index, columns = df.columns)
    return temp_df


## Set up

In [3]:

path2feature = 'Data/feature_data_all'

path_to_file = 'Data/df_train_sales_filled.csv' #imputed sales

#path_to_file ='sample_filled.csv' #raw

df = pd.read_csv(path_to_file,index_col=0)
df.columns = pd.to_datetime(df.columns)


Model ='M2Q' #Choice of model
is_far=0
if is_far == 1:
    far = 'Far'
else:
    far= 'Near'

#quantile_list = [50,60]
quantile_list = [50,60,70,80,90,95]  # changed from q_str_list
#pred_len_list=[14]
pred_len_list = [91, 31, 14, 7, 3, 1]


#q_str_list = ['{:.2f}'.format(i) for i in q_list]
days_per_train = 1 # fixed
# if Model =='Q2Q':
#     is_far = 1 
# elif Model =='M2Q':
#     is_far = 0 
   
    
quantile_window_list=[7,14,28,56,112]
mean_window_list=[3,7,14,28,56,112]

# Hyperparameter grid
param_grid = {'learning_rate': [0.05], 'n_estimators':[200],'num_leaves': [31]               , 'bagging_freq':[1], 'bagging_fraction':[0.8]               , 'feature_fraction':[0.8]}

# param_grid = {'learning_rate': [0.05, 0.03, 0.01], 'n_estimators':[200] \
#               ,'num_leaves': [31, 61, 91, 127] \
#               , 'bagging_freq':[1], 'bagging_fraction':[0.8] \
#               , 'feature_fraction':[1, 0.8 ,0.6]}
# param_grid = {'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3], 'n_estimators':[100, 150, 200, 250, 300]}

# Cross validation
cv=5


## Read in preprocessed features

In [4]:
tick = datetime.now()
quantile_feature, mean_feature = read_feature_data(quantile_list, quantile_window_list, mean_window_list, path2feature)

print datetime.now()-tick


Reading  window_7_quantile_50
Reading  window_7_quantile_60
Reading  window_7_quantile_70
Reading  window_7_quantile_80
Reading  window_7_quantile_90
Reading  window_7_quantile_95
Reading  window_14_quantile_50
Reading  window_14_quantile_60
Reading  window_14_quantile_70
Reading  window_14_quantile_80
Reading  window_14_quantile_90
Reading  window_14_quantile_95
Reading  window_28_quantile_50
Reading  window_28_quantile_60
Reading  window_28_quantile_70
Reading  window_28_quantile_80
Reading  window_28_quantile_90
Reading  window_28_quantile_95
Reading  window_56_quantile_50
Reading  window_56_quantile_60
Reading  window_56_quantile_70
Reading  window_56_quantile_80
Reading  window_56_quantile_90
Reading  window_56_quantile_95
Reading  window_112_quantile_50
Reading  window_112_quantile_60
Reading  window_112_quantile_70
Reading  window_112_quantile_80
Reading  window_112_quantile_90
Reading  window_112_quantile_95
Reading  window_3_mean
Reading  window_7_mean
Reading  window_14_mean


In [ ]:
## Batch run
tmp_date_list = list(map(lambda date: date.strftime('%Y-%m-%d'), [date(2017,4,1) + relativedelta(months=i) for i in range(14)])) # 2017-4 ~ 2018-5 
tmp_date_list = list(map(lambda date: date.strftime('%Y-%m-%d'), [date(2017,4,1)])) # 2017-4 ~ 2018-5 



for i, tmp_date in enumerate(tmp_date_list):
    seed = 1988
    train_model = {} # Model: dim(q) * dim(pred_len) = 6 * 6
    train_pred = {}
    train_true = {}
    start_model_train_time=datetime.now()

    pred_date = datetime.strptime(tmp_date, '%Y-%m-%d')    
    test_date = pred_date - timedelta(days=days_per_train)
    
  
    Model_output = 'Model_'+Model+'_'+far+'_'+tmp_date+'.pl'
    Train_output = 'Train_'+Model+'_'+far+'_'+tmp_date+'.pl'
    Prediction_output = 'Pred_'+Model+'_'+far+'_'+tmp_date+'.pl' 
    
    gb_df_pred_list = pd.DataFrame()
    gb_df_train_pred_list = pd.DataFrame() # Training prediction
    gb_df_train_true_list = pd.DataFrame() # Training y   

    print 'Training model on data on and before:', test_date

    for pred_len in pred_len_list:
        
        df_pred_list = pd.DataFrame()
        df_train_pred_list = pd.DataFrame() # Training prediction
        df_train_true_list = pd.DataFrame() # Training y

        for q in quantile_list:
            
            temp_name = 'q'+str(q)+'len'+str(pred_len) 
            print "Training ",Model,'_',far,'_',temp_name 

            if Model =='M2Q':
#                 estimator = lgb.LGBMRegressor(random_state=seed, objective='quantile', alpha=q*0.01)
                estimator = lgb.LGBMRegressor(random_state=seed, objective='quantile', alpha=q*0.01 \
                                             ,learning_rate =0.05, n_estimators=200, num_leaves = 31 \
                                              , bagging_freq=1, num_threads =8, bagging_fraction=0.8, feature_fraction=0.8)
#                   estimator = lgb.LGBMRegressor(random_state=seed, objective='quantile', alpha=q*0.01 \
#                                              ,learning_rate =0.05, n_estimators=200, num_leaves = 31)
            elif Model =='Q2Q':
                estimator = lgb.LGBMRegressor(random_state=seed, alpha=q*0.01\
                                             ,learning_rate =0.05, n_estimators=200, num_leaves = 31 \
                                              , bagging_freq=1, num_threads =8, bagging_fraction=0.8, feature_fraction=0.8)

#### Train ###
            tic = datetime.now()
            X_train, y_train = prepare_training_data(df, q, is_far, pred_len, Model, test_date, mean_window_list, quantile_window_list, is_train=True)
            X_train = np.round(X_train,4)
            y_train = np.round(y_train,4)
            
            estimator.fit(X_train, y_train)
            train_model[temp_name] = estimator
            
            print "Predicting ",Model,'_',far,'_',temp_name,' for:',pred_date
            
            X_test= prepare_training_data(df, q, is_far, pred_len, Model, pred_date, mean_window_list, quantile_window_list, is_train=False)
            X_test = np.round(X_test, 4)
            y_test = estimator.predict(X_test)**2 # y_train has been transformed **(1/2)
            y_test = np.round(y_test,4)
            
            df_pred = pd.DataFrame(y_test, index=X_test.index)
            y_test = np.array(y_test).transpose()
            df_pred = pd.DataFrame(y_test, index=X_test.index, columns=[(q,pred_len)])
            df_pred[(q,pred_len)] = df_pred[(q,pred_len)].apply(lambda x: np.max([x,0]))
                        
            if len(df_pred_list)==0:
                df_pred_list = df_pred
            else:
                df_pred_list = pd.concat([df_pred_list, df_pred], axis=1)
            
            ## Training Performances
            print 'Generating training performance.'
            y_train_pred = estimator.predict(X_train)**2
            y_train_true = np.array(y_train)**2
            
            df_train_pred = pd.DataFrame(y_train_pred, index=X_train.index)
            df_train_true = pd.DataFrame(y_train_true, index=X_train.index)
            
            y_train_pred = np.array(y_train_pred).transpose()
            y_train_true = np.array(y_train_true).transpose()
            
            df_train_pred = pd.DataFrame(y_train_pred, index=X_train.index, columns=[(q,pred_len)]).sort_index()
            df_train_true = pd.DataFrame(y_train_true, index=X_train.index, columns=[(q,pred_len)]).sort_index()
            
            df_train_pred[(q,pred_len)] = df_train_pred[(q,pred_len)].apply(lambda x: np.max([x,0]))
            df_train_true[(q,pred_len)] = df_train_true[(q,pred_len)].apply(lambda x: np.max([x,0]))
                        
            if len(df_train_pred_list)==0:
                df_train_pred_list = df_train_pred
                df_train_true_list = df_train_true
            else:
                df_train_pred_list = pd.concat([df_train_pred_list, df_train_pred], axis=1)
                df_train_true_list = pd.concat([df_train_true_list, df_train_true], axis=1)
                        
            
            del X_test, y_test, X_train, y_train, df_train_pred, df_train_true, df_pred
            
        # Sorted by quantile        
        df_pred_list_sort = sort_quantile(df_pred_list)        
        df_train_pred_list_sort = sort_quantile(df_train_pred_list)        
        df_train_true_list_sort = sort_quantile(df_train_true_list)
                        
        if len(gb_df_pred_list)==0:
            gb_df_pred_list = df_pred_list_sort 
            gb_df_train_pred_list = df_train_pred_list_sort
            gb_df_train_true_list = df_train_true_list_sort
        else:
            gb_df_pred_list =pd.concat([gb_df_pred_list, df_pred_list_sort], axis=1)
            gb_df_train_pred_list = pd.concat([gb_df_train_pred_list, df_train_pred_list_sort], axis=1)
            gb_df_train_true_list = pd.concat([gb_df_train_true_list, df_train_true_list_sort], axis=1)
        
        del df_pred_list, df_train_pred_list, df_train_true_list
        
    print 'Total training time is: ', datetime.now()-start_model_train_time 

# # ## For batch run    
    # Save Model
    file = open(os.path.join('Experiment/Baseline/Model',Model_output), 'wb')
    pickle.dump(train_model, file)
    file.close()    
    # Save Training
    file = open(os.path.join('Experiment/Baseline/Train',Train_output), 'wb')
    pickle.dump(gb_df_train_pred_list, file)
    pickle.dump(gb_df_train_true_list, file)
    file.close()
    
    # Save Prediction
    file = open(os.path.join('Experiment/Baseline/Predict',Prediction_output), 'wb')
    pickle.dump(gb_df_pred_list, file)
    file.close()
    

Training model on data on and before: 2017-03-31 00:00:00
Training  M2Q _ Near _ q50len91
Predicting  M2Q _ Near _ q50len91  for: 2017-04-01 00:00:00
Generating training performance.
Training  M2Q _ Near _ q60len91
Predicting  M2Q _ Near _ q60len91  for: 2017-04-01 00:00:00
Generating training performance.
Training  M2Q _ Near _ q70len91
Predicting  M2Q _ Near _ q70len91  for: 2017-04-01 00:00:00
Generating training performance.
Training  M2Q _ Near _ q80len91
Predicting  M2Q _ Near _ q80len91  for: 2017-04-01 00:00:00
Generating training performance.
Training  M2Q _ Near _ q90len91
Predicting  M2Q _ Near _ q90len91  for: 2017-04-01 00:00:00
Generating training performance.
Training  M2Q _ Near _ q95len91
Predicting  M2Q _ Near _ q95len91  for: 2017-04-01 00:00:00
Generating training performance.
Training  M2Q _ Near _ q50len31
Predicting  M2Q _ Near _ q50len31  for: 2017-04-01 00:00:00
Generating training performance.
Training  M2Q _ Near _ q60len31
Predicting  M2Q _ Near _ q60len31  

In [26]:
file = open('Data/X_train.pl','rb')
X_train_server = pickle.load(file)
X_train_round_server = pickle.load(file)
file.close()

In [27]:
X_train.equals(X_train_server)

False

In [28]:
X_train_round.equals(X_train_round_server)

True

In [31]:
file = open('Data/pred.pl','rb')
df_pred_server = pickle.load(file)

file.close()

In [32]:
df_pred.equals(df_pred_server)

False

In [7]:
file = open('Data/test_bagging_4threads.pl','rb')
X_test_server = pickle.load(file)
y_test_server = pickle.load(file)
file.close()

In [8]:
X_test.equals(X_test_server)

True

In [9]:
np.array_equal(y_test,y_test_server)

True

In [49]:
sum(y_test - y_test_server)

-10530.975100000094